# Eat Safe, Love

## Notebook Set Up

In [1]:
from pymongo import MongoClient
import pandas as pd
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
print(db.list_collection_names())

['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']
print(establishments)

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'uk_food'), 'establishments')


In [6]:
result = establishments.find().limit(5)  # Limit to the first 5 documents
for doc in result:
    pprint(doc)

{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': '1',
 'Distance': 4623.972328074718,
 'LocalAuthorityCode': '511',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'NewRatingPending': True,
 'Phone': '',
 'PostCode': 'SE18 7DY',
 'RightToReply': '',
 'SchemeType': 'FHRS',
 '_id': ObjectId('6774402ebb99defad809698c'),
 'geocode': {'latitude': {'$toDouble': '$geocode.latitude'},
             'longitude': {'$toDouble': '$geocode.longitude'}},
 'scores': {'ConfidenceInManagement': '',
            'Hygiene': {'$toInt': '$scores.Hygiene'},
            'Structural': ''}}
{'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'BusinessName': 'Pen

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [7]:
# Find the establishments with a hygiene score of 20
query = {"scores.Hygiene": 20}

# Use count_documents to display the number of documents in the result
document_count = establishments.count_documents(query)
print("Number of documents in Hygiene score: ", document_count)

# Display the first document in the results using pprint
result = establishments.find_one(query)
pprint(result)


Number of documents in Hygiene score:  0
None


In [8]:
# Convert the result to a Pandas DataFrame
hygiene_20 = establishments.find(query)
result_df = pd.DataFrame(hygiene_20)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(result_df))
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:


In [9]:
result_df.head(10)


""


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [10]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query ={
    "RatingValue": {"$gte": 4},
    "LocalAuthorityName": {"$regex": "London", "$options": "i"}
}
# Use count_documents to display the number of documents in the result
london_docrating_count = establishments.count_documents(query)

# Display the first document in the results using pprint
london_docrating_first = establishments.find_one(query)
pprint(london_docrating_first)

None


In [11]:
# Convert the result to a Pandas DataFrame
london_results = establishments.find(query)
london_results_df = pd.DataFrame(list(london_results))

# Display the number of rows in the DataFrame
print(f"Rows in Data Frame: {len(london_results_df)}")

# Display the first 10 rows of the DataFrame
london_results_df.head(10)


Rows in Data Frame: 0


""


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.490142
longitude = 0.08384000

query = {
    'geocode.longitude': {'$gte':longitude-degree_search, '$lte':longitude+degree_search},
    'geocode.latitude': {'$gte':latitude-degree_search, '$lte':latitude+degree_search},
    'RatingValue': {'$gte': 5}
    }
sort = [("Hygiene", 1)]
limit = 5

# Print the results
pprint(list(establishments.find(query).sort(sort).limit(limit)))


[]


In [13]:
# Convert result to Pandas DataFrame
top5_df = pd.DataFrame(establishments.find(query).sort(sort).limit(limit))
top5_df.head()


""


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that:
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest
pipeline = [
    {'$match': {'scores.Hygiene': 0}},
    {'$group': {'_id': '$LocalAuthorityName', 'count': {'$sum': 1}}},
    {'$sort': {'count': -1}}
]

results = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
print(f'Number of documents: {len(results)}')
# Print the first 10 results
pprint(results[0:10])

Number of documents: 0
[]


In [15]:
# Convert the result to a Pandas DataFrame
df_results = pd.DataFrame(results)

# Display the number of rows in the DataFrame
print(f"Number of rows in the DataFrame: {len(df_results)}")

# Display the first 10 rows of the DataFrame
df_results.head(10)


Number of rows in the DataFrame: 0


""
